In [1]:
import tensorflow as tf
import numpy as np
import os
from utils import *
from sklearn.preprocessing import MultiLabelBinarizer
import datetime
import pandas as pd

from tensorflow.python.saved_model import tag_constants


import tensorflow.contrib.slim as slim

from apc import AveragePrecisionCalculator

import matplotlib.pyplot as plt


/home/ec2-user/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
import hashlib
import itertools
import json
import os
from six.moves import urllib
import sys

In [ ]:
tf.reset_default_graph()

In [3]:
def build_graph(tstep_size=1e-3):
    xdim = 1024+128
    ydim = 3862
    X = tf.placeholder(tf.float32, shape=[None, xdim], name='X')
    Y = tf.placeholder(tf.float32, shape=[None, ydim], name='Y')
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')

    
    x_img, x_audio = tf.split(X, [1024, 128], axis=-1)

    img_reshape = tf.reshape(x_img, [-1, 32, 32, 1])

    h1 = lrelu(conv2d(img_reshape, 1, 16, 'conv_1')) # 32x32x1 -> 16x16x16
    h2 = conv2d_with_pooling(h1, 16, 32, 'conv_2')# 16x16x16 -> 8x8x32
    h2_flat = tf.reshape(h2,[-1, 8*8*32])
    h1_flat = tf.reshape(h1,[-1, 16*16*16])

    concat = tf.concat([h1_flat, h2_flat, x_audio], 1)
    h = lrelu(dense(concat, 16*16*16 + 8*8*32 + 128, 100, scope='h'))
    pred = dense(h, 100, ydim, scope = 'pred')

    
    loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=pred)
    mloss = tf.reduce_mean(loss)

    train = tf.train.AdamOptimizer(tstep_size).minimize(loss)
    
    return X, Y, keep_prob, pred, loss, mloss, train

In [4]:
X, Y, keep_prob, pred, loss, mloss, train = build_graph(tstep_size=1e-3)


In [4]:
# GRAPH CONV
xdim = 1024+128
ydim = 3862
X = tf.placeholder(tf.float32, shape=[None, xdim], name='X')
Y = tf.placeholder(tf.float32, shape=[None, ydim], name='Y')
keep_prob = tf.placeholder(tf.float32, name='keep_prob')

x_img, x_audio = tf.split(X, [1024, 128], axis=-1)

img_reshape = tf.reshape(x_img, [-1, 32, 32, 1])

h1 = lrelu(conv2d(img_reshape, 1, 16, 'conv_1')) # 32x32x1 -> 16x16x16
h2 = conv2d_with_pooling(h1, 16, 32, 'conv_2')# 16x16x16 -> 8x8x32
h2_flat = tf.reshape(h2,[-1, 8*8*32])
h1_flat = tf.reshape(h1,[-1, 16*16*16])

concat = tf.concat([h1_flat, h2_flat, x_audio], 1)
h = lrelu(dense(concat, 16*16*16 + 8*8*32 + 128, 100, scope='h'))
pred = dense(h, 100, ydim, scope = 'pred')

loss = tf.nn.sigmoid_cross_entropy_with_logits(labels=Y, logits=pred)
mloss = tf.reduce_mean(loss)

train = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [5]:
# INIT
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [ ]:
def train_iter(b, sess, mloss, X, Y, keep_prob, ):
    for i in range(75):
        batch = b.next_batch(32)
        if i % 74 == 0:
            loss = sess.run(mloss, feed_dict={X: b.x, Y: b.y, keep_prob:1.0})
            print('step %d, loss %g' % (i, loss))
            if i == 0:
                loss_plt.append(loss)
        sess.run(train, feed_dict={X: batch[0], Y: batch[1], keep_prob:0.5})

In [ ]:
class file_iterator:
    def __init__(self):
        tempfile = '/home/ec2-user/data/yt8m/test_all/xx.tfrecord'
        mirror = 'us'
        partition = '2/video/train'
        partition_parts = partition.split('/')
        plan_url = 'http://data.yt8m.org/{}/download_plans/{}_{}.json'.format(
              partition_parts[0], partition_parts[1], partition_parts[2])
        download_plan = json.loads(urllib.request.urlopen(plan_url).read())
        files = [f for f in download_plan['files'].keys()]

        ct = 0
        start = datetime.datetime.now()
        loss_plt = []
        
    

In [6]:
# TRAIN
one_hot = MultiLabelBinarizer(classes=[i for i in range(3862)])

tempfile = '/home/ec2-user/data/yt8m/test_all/xx.tfrecord'
mirror = 'us'
partition = '2/video/train'
partition_parts = partition.split('/')
plan_url = 'http://data.yt8m.org/{}/download_plans/{}_{}.json'.format(
      partition_parts[0], partition_parts[1], partition_parts[2])
download_plan = json.loads(urllib.request.urlopen(plan_url).read())
files = [f for f in download_plan['files'].keys()]

ct = 0
start = datetime.datetime.now()
loss_plt = []
for f in files[1:400]:
    ct+=1
    print('starting file {} of {}'.format(ct, len(files)))
    print('its been this much time: {}'.format(datetime.datetime.now()-start))
    print('{} % done'.format(int(ct/len(files)*100)))
    try:
        print('estimated time remaining {}'.format( (datetime.datetime.now()-start)/ ((ct-1)/(len(files)-(ct-1))) ))
    except:
        pass
    print(f)
    
    s=  datetime.datetime.now()
    download_url = 'http://%s.data.yt8m.org/%s/%s' % (mirror, partition, f)
    download_file(download_url, tempfile)
    
    print('download time: {}'.format(datetime.datetime.now()-s))
    
    vid_ids, labels, mean_rgb, mean_audio = get_data_from_file(tempfile)
    
    onehot_labels = one_hot.fit_transform(labels)
    b = BatchIterator(onehot_labels, mean_rgb,mean_audio)
    # TRAIN
    for i in range(75):
        batch = b.next_batch(32)
        if i % 74 == 0:
            loss = sess.run(mloss, feed_dict={X: b.x, Y: b.y, keep_prob:1.0})
            print('step %d, loss %g' % (i, loss))
            if i == 0:
                loss_plt.append(loss)
        sess.run(train, feed_dict={X: batch[0], Y: batch[1], keep_prob:0.5})

    print('')

starting file 1 of 3844
its been this much time: 0:00:00.000384
0 % done
trainKH.tfrecord
Succesfully downloaded /home/ec2-user/data/yt8m/test_all/xx.tfrecord 4772265 bytes.
download time: 0:00:00.334933
step 0, loss 0.692879


KeyboardInterrupt: 

In [ ]:
plt.plot(loss_plt[300:])

In [7]:
# SAVE

export_dir = '/home/ec2-user/Notebooks/MyModel18/'

builder = tf.saved_model.builder.SavedModelBuilder(export_dir)

sig_def = tf.saved_model.signature_def_utils.build_signature_def(
                                        inputs= {"X": tf.saved_model.utils.build_tensor_info(X), 
                                                 "Y":tf.saved_model.utils.build_tensor_info(Y), 
                                                 "keep_prob":tf.saved_model.utils.build_tensor_info(keep_prob)},
                                        outputs= {"pred": tf.saved_model.utils.build_tensor_info(pred)})

builder.add_meta_graph_and_variables(sess,
                                  ['tag'],
                                     signature_def_map= {"model":sig_def}        
                                    )
builder.save()

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: /home/ec2-user/Notebooks/MyModel18/saved_model.pb


b'/home/ec2-user/Notebooks/MyModel18/saved_model.pb'

# TRAIN
one_hot = MultiLabelBinarizer(classes=[i for i in range(3862)])
datadir = '/home/ec2-user/data/yt8m/train/'

# Iterate through train directory
for file in os.listdir(datadir)[:13]:
    if file[-5:]=='.json':
        continue
    filename = datadir + file
    vid_ids, labels, mean_rgb, mean_audio = get_data_from_file(filename)

    onehot_labels = one_hot.fit_transform(labels)
    #onehot_labels = onehot_labels[:,:ydim]
    
    b = BatchIterator(onehot_labels, mean_rgb,mean_audio)
    # TRAIN
    for i in range(1001):
        batch = b.next_batch(32)
        if i % 500 == 0:
            loss = sess.run(mloss, feed_dict={X: batch[0], Y: batch[1], keep_prob:1.0})
            print('step %d, loss %g' % (i, loss))
        sess.run(train, feed_dict={X: batch[0], Y: batch[1], keep_prob:0.5})

    print('')
    
    
    # print('test accuracy %g' % sess.run(accuracy, 
    #                                     feed_dict={X: mnist.test.images, y_: mnist.test.labels}))


In [ ]:
#EVALUATE
datadir = '/home/ec2-user/data/yt8m/validate/'
APC = AveragePrecisionCalculator()
appp = []
for file in os.listdir(datadir)[:20]:
    if file[-5:]=='.json':
        continue
    filename = datadir + file    
    vid_ids, labels, mean_rgb, mean_audio = get_data_from_file(filename)

    onehot_labels = one_hot.fit_transform(labels)
    #onehot_labels = onehot_labels[:,:869]
    
    b = BatchIterator(onehot_labels, mean_rgb,mean_audio)
    
    p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x, Y: b.y, keep_prob:1.0})

    ap = 0
    for i in range(p.shape[0]):
        ap+=APC.ap(p[i], b.y[i])

    ap_avg = ap/p.shape[0]
    print(ap_avg)
    appp.append(ap_avg)
print('average: {}'.format(np.mean(ap_avg)))

#INFER
datadir = '/home/ec2-user/data/yt8m/test/'
outfile = 'out/'+ str(datetime.datetime.now())+'.csv'
with open(outfile, 'w') as f:
    pd.DataFrame(columns=['VideoIds','LabelConfidencePairs']).to_csv(f, index=False)
    
for file in os.listdir(datadir)[:3]:
    if file[-5:]=='.json':
        continue
    filename = datadir + file

    print(file)
    
    vid_ids, _, mean_rgb, mean_audio = get_data_from_file(filename,has_labels=False)
    
    fake_labels = [[1] for _ in range(len(mean_rgb))]
    
    b = BatchIterator(fake_labels, mean_rgb,mean_audio)
    
    p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x, keep_prob:1.0})
    
    top20 = p.argsort(axis=-1)[:,-20:][:,::-1]
    x_axis_index=np.tile(np.arange(len(p)), (top20.shape[1],1)).transpose() 
    probs = p[x_axis_index, top20]
    
    idx_probs = np.stack([top20, probs], axis=-1)
    idx_probs = [[(int(idx_probs[i,j,0]), idx_probs[i,j,1]) for j in range(idx_probs.shape[1])] for i in range(idx_probs.shape[0])]
    
    out = [str(idx_probs[0]).replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace(',', '') for i in range(len(idx_probs))]
    
    df = pd.DataFrame(data={'VideoId':vid_ids, 'LabelConfidencePairs':out})
    with open(outfile, 'a') as f:
        df.to_csv(f, header=False, index=False)
    
    
    

In [ ]:
datadir = '/home/ec2-user/data/yt8m/test/'
tempfile = '/home/ec2-user/data/yt8m/test_all/xx.tfrecord'
outfile = 'out/'+ str(datetime.datetime.now())+'.csv'

with open(outfile, 'w') as f:
    pd.DataFrame(columns=['VideoId','LabelConfidencePairs']).to_csv(f, index=False)
    
mirror = 'us'
partition = '2/video/test'
partition_parts = partition.split('/')
plan_url = 'http://data.yt8m.org/{}/download_plans/{}_{}.json'.format(
      partition_parts[0], partition_parts[1], partition_parts[2])
download_plan = json.loads(urllib.request.urlopen(plan_url).read())
files = [f for f in download_plan['files'].keys()]

ct = 0
start = datetime.datetime.now()
for f in files:
    ct+=1
    print('starting file {} of {}'.format(ct, len(files)))
    print('its been this much time: {}'.format(datetime.datetime.now()-start))
    print('{} % done'.format(int(ct/len(files)*100)))
    try:
        print('estimated time remaining {}'.format( (datetime.datetime.now()-start)/ ((ct-1)/(len(files)-(ct-1))) ))
    except:
        pass
    print(f)
    
    download_url = 'http://%s.data.yt8m.org/%s/%s' % (mirror, partition, f)
    download_file(download_url, tempfile)
    
    vid_ids, _, mean_rgb, mean_audio = get_data_from_file(tempfile, has_labels=False)
    
    fake_labels = [[1] for _ in range(len(mean_rgb))]
    b = BatchIterator(fake_labels, mean_rgb,mean_audio)
    
    p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x, keep_prob:1.0})
    
    top20 = p.argsort(axis=-1)[:,-20:][:,::-1]
    x_axis_index=np.tile(np.arange(len(p)), (top20.shape[1],1)).transpose() 
    probs = p[x_axis_index, top20]
    idx_probs = np.stack([top20, probs], axis=-1)
    idx_probs = [[(int(idx_probs[i,j,0]), idx_probs[i,j,1]) for j in range(idx_probs.shape[1])] for i in range(idx_probs.shape[0])]
    out = [str(idx_probs[i]).replace('[', '').replace(']', '').replace('(', '').replace(')', '').replace(',', '') for i in range(len(idx_probs))]
    df = pd.DataFrame(data={'VideoId':vid_ids, 'LabelConfidencePairs':out})
    
    with open(outfile, 'a') as f:
        df.to_csv(f, header=False, index=False)

In [ ]:
p = sess.run(tf.sigmoid(pred), feed_dict={X: b.x, Y: b.y})

predicted = one_hot.inverse_transform(np.concatenate([np.array(p>.5, dtype=int), np.zeros([292,3862-869])], axis=1))
actual = one_hot.inverse_transform(np.concatenate([b.y, np.zeros([292,3862-869])], axis=1))




In [ ]:
correct=0

for i in range(len(actual)):
    if actual[i]==predicted[i]:
        correct+=1
print(correct/len(actual))

In [ ]:
actual[0]==predicted[0]

In [ ]:
actual

In [ ]:
predicted

In [ ]:
p.shape

In [ ]:
one_hot.inverse_transform(np.concatenate([batch[1], np.zeros([32,3862-869])], axis=1))

In [ ]:
one_hot.classes[:869][batch[1][0]>.5]

In [ ]:
one_hot.classes

In [ ]:
np.zeros(2)[[True,False]]

In [ ]:
datadir = '/home/ec2-user/data/yt8m/test/'

In [ ]:
file=os.listdir(datadir)[5]
filename = datadir + file
for example in tf.python_io.tf_record_iterator(filename):
    tf_example = tf.train.Example.FromString(example)
    vid_ids, labels, mean_rgb, mean_audio = get_data_from_file(filename)

In [ ]:
def get_data_from_file(filename, has_labels=True):
    vid_ids, labels, mean_rgb, mean_audio = [], [], [], []

    for example in tf.python_io.tf_record_iterator(filename):
        tf_example = tf.train.Example.FromString(example)

        vid_ids.append(tf_example.features.feature['id'].bytes_list.value[0].decode(encoding='UTF-8'))
        labels.append(tf_example.features.feature['labels'].int64_list.value)
        mean_rgb.append(tf_example.features.feature['mean_rgb'].float_list.value)
        mean_audio.append(tf_example.features.feature['mean_audio'].float_list.value)
    return vid_ids, labels, mean_rgb, mean_audio

In [ ]:
labels[-1]

In [ ]:
tf_example

In [ ]:
p = sess.run(tf.sigmoid(pred),
        feed_dict={X: batch[0], Y: batch[1]})

In [ ]:
np.sum(p>.5, axis=1)

In [ ]:
one_hot.classes_[p[6]>.5]

In [ ]:
yy = batch[1][6]

In [ ]:
one_hot.classes_[yy>.5]

In [ ]:
yy

In [ ]:
import apc_file

In [ ]:
apcc = apc_file.AveragePrecisionCalculator()

In [ ]:
apcc.ap(p[2], b.y[2])

In [ ]:
p